## PS1: Optimize the feature set of a rack-mounted M2 MacPro Server
The [new MacPro with the M2 Ultra chip has been released](https://www.apple.com/shop/buy-mac/mac-pro/rack#). The MacPro M2 has several configuration options broadly organized into five categories: `{CPU, Memory, Storage, Accessories, Software}` with multiple options per category:
* The `CPU` category has `2` options. Only one option can be selected from the `CPU` category
* The `Memory` category has `3` options. Only one option can be selected from the `Memory` category.
* The `Storage` category has `4` options. Only one option can be selected from the `Storage` category.
* The `Accessories` category has `3` options. Only one option can be selected from the `Accessories` category
* The `Software` category has `2` options. Neither or both options can be selected from the `Software` category

### Problem statement
Using a `linear` utility model and budget and feature constraints, compute the optimal set of features for a rack-mounted MacPro M2. This problem will be similar to the examples discussed in the lecture and discussion, except for one crucial element: the decision variables (our choices) will be `binary`, i.e., $x_{i}\in{0,1}$ where a `0` indicates that we do NOT choose feature $i$. In contrast, a value of `1` indicates that we choose option $i$. 

Formally, an agent has a set of $n$ configuration options $X = \left\{x_{i}\right\}_{i=1}^{n}$, a `Linear` utility function, and a total of $I$ units of resource to allocate, e.g., money, and potentially other constraints. An optimal agent maximizes its utility subject to its resource and other constraints:

\begin{eqnarray}
\text{maximize}~\mathcal{O} &=& \sum_{i\in{1,\dotsc,n}}\alpha_{i}x_{i} \\
\text{subject to}~\sum_{i\in{1,\dotsc,n}}c_{i}x_{i} & = & I\\
\text{and}~\mathbf{C}\mathbf{x} & \leq & \mathbf{b} \\
\text{and}~x_{i}&\in&{0,1}\qquad{i=1,2,\dots,n}
\end{eqnarray}

The quantity $c_{i}\geq{0}~\forall{i}$ denotes the cost of option $i$, $\alpha_{i}$ denotes user-specified coefficient in the `Linear` utility function, $x_{i}\in{0,1}$ represents the choice of option $i$, and $\mathbf{C}\mathbf{x} \leq \mathbf{b}$ represents additional constraints governing the decision.

#### List of Tasks
* __Task 1__: Specify $\alpha$- and $c$-vectors for the problem
* __Task 2__: Specify the additional constraint matrix $\mathbf{C}$
* __Task 3__: Specify the problem object and solve the problem
* __Task 4__: Try at least two different weighting schemes and different budget values, and explore how these design choices influence the optimal configuration (one of these __can__ be the `default` values specified below).

### Setup
The computations in this problem set rely on [VLDecisionsPackage.jl](https://github.com/varnerlab/VLDecisionsPackage.jl.git) and several external `Julia` packages. To load the required packages and any custom codes the teaching team has developed to work with these packages, we [include](https://docs.julialang.org/en/v1/manual/code-loading/) the `Include.jl` file):

In [1]:
include("Include.jl");

  Activating project at `~/Documents/GitHub/CHEME-5760-Labs-F23`
Precompiling project...
  ✓ SnoopPrecompile
  ✓ Graphite2_jll
  ✓ OpenSSL_jll
  ✓ LLVMOpenMP_jll
  ✓ Rmath_jll
  ✓ Bzip2_jll
  ✓ RecipesBase
  ✓ libpng_jll
  ✓ libfdk_aac_jll
  ✓ GLPK_jll
  ✓ StringManipulation
  ✓ LAME_jll
  ✓ LERC_jll
  ✓ fzf_jll
  ✓ JpegTurbo_jll
  ✓ XZ_jll
  ✓ Ogg_jll
  ✓ x265_jll
  ✓ libaom_jll
  ✓ x264_jll
  ✓ Arpack_jll
  ✓ Zstd_jll
  ✓ Expat_jll
  ✓ LZO_jll
  ✓ Opus_jll
  ✓ Libiconv_jll
  ✓ Libffi_jll
  ✓ FFTW_jll
  ✓ OpenSpecFun_jll
  ✓ FriBidi_jll
  ✓ Libuuid_jll
  ✓ libinput_jll
  ✓ InlineStrings
  ✓ Pixman_jll
  ✓ BenchmarkTools
  ✓ Rmath
  ✓ CodecBzip2
  ✓ FreeType2_jll
  ✓ libvorbis_jll
  ✓ JLFzf
  ✓ Arpack
  ✓ OpenSSL
  ✓ XML2_jll
  ✓ Libtiff_jll
  ✓ WeakRefStrings
  ✓ Fontconfig_jll
  ✓ MultivariateStats
  ✓ SpecialFunctions
  ✓ Gettext_jll
  ✓ XSLT_jll
  ✓ Wayland_jll
  ✓ HTTP
  ✓ SpecialFunctions → SpecialFunctionsChainRulesCoreExt
  ✓ DiffRules
  ✓ DualNumbers
  ✓ ColorVectorSpace → Spe

#### Specify constants and other static stuff

In [2]:
number_of_choices = 14;
bounds_array = Array{Float64,2}(undef, number_of_choices,2)
for i ∈ 1:number_of_choices
    bounds_array[i,1] = 0.0
    bounds_array[i,2] = 1.0;
end

## Task 1: Specify the configuration array
The `configuration_array` is a `14` $\times$ `2` array holding perception and cost information about the problem. Each row of the `configuration_array` contains data for a particular MacPro configuration option. The first column contains the coefficients of the `Linear` utility function, i.e., the elements of the $\alpha$-vector, while the unit price of the features, i.e., elements of the $c$-vector, are in the second column.  

#### Feature prices
The prices of each configuration option have been estimated from the [Apple MacPro server website](https://www.apple.com/shop/buy-mac/mac-pro/rack#).

#### Linear utility weighting scheme
In `PS1`, we'll use a category-based weighting scheme. In each of the five categories, allocate `1` unit of weight:

* In each category, the coefficients in the `Linear` utility function for options in that category must sum to one.

For example, if you have equal feelings about three options in a category, then `0.33, 0.33, 0.33` would be a typical scheme. On the other hand, if you are excited about feature `1` over the other two in the category, then `0.8,0.1,0.1` could be an appropriate weight scheme.

In [3]:
configuration_array = [

    # CPU options
    0.5 2640.0    ; # 1 CPU 1
    0.5 3649.0    ; # 2 CPU 2

    # Memory options
    0.333 3840.0  ; # 3 Memory 1
    0.333 4640.0  ; # 4 Memory 2
    0.333 3600.0  ; # 5 Memory 3

    # Storage options -
    0.25 1440.0   ; # 6 Storage 1
    0.25 1840.0   ; # 7 Storage 2
    0.25 2440.0   ; # 8 Storage 3
    0.25 3640.0   ; # 9 Storage 4

    # Accessory options
    0.333 79.0    ; # 10 Accessory 1
    0.333 129.0   ; # 11 Accessory 2
    0.333 149.0   ; # 12 Accessory 3
    
    # Software options
    0.5 299.0     ; # 13 Software 1
    0.5 149.0     ; # 14 Software 2
];

### Task 2: Specify the feature constraint matrix `C`
In each category, only a finite number of options can be selected simultaneously, typically only a single option, with the exception being the `Software` category, which is unconstrained (can have from zero up to two items selected). Because the decision variables are binary, we can implement this requirement with an additional set of constraints of the form:

$$
\begin{equation}
\mathbf{C}\mathbf{x} = \mathbf{1}
\end{equation}
$$

where $\mathbf{C}$ denotes the constraint matrix, $\mathbf{x}$ denotes the choice vector and $\mathbf{1}$ denotes a vector of `1`'s. 
Specify the $\mathbf{C}$ matrix for this problem:

In [13]:
C = zeros(4,14); 

# understanding qestion: why is this a 4 x 14 array?
# because there are 4 categories: CPU, Memory, Storage, Accessory (excluding Software because it's unconstrained)
# for each of these four constrained groups (rows), we need 14 columns to specify which of those 14 options (features) belong to its category

C[1,1:2] .= 1;
C[2,3:5] .= 1;
C[3,6:9] .= 1;
C[4,10:12] .= 1;

__Note__: Julia's Array syntax is similar to Matlab/Octave, except with square brackets. See [the Array documentation](https://docs.julialang.org/en/v1/base/arrays/) or various other [Julia tutorials on the web](https://www.tutorialspoint.com/julia/julia_arrays.htm) about working with the Array data structure.

### Task 3: Specify the problem object, and solve for the optimal configuration
Finally, build a `MySimpleBinaryVariableLinearChoiceProblem` model using the `build(...)` method, set this instance to the `problem` variable, and then pass the `problem` object to the `solve(...)` method. The `solve(...)` method will solve the `ILP` problem using the [GLPK.jl](https://github.com/jump-dev/GLPK.jl) interface to the [GLPK linear programming solver](https://www.gnu.org/software/glpk/).

In [18]:
α = configuration_array[:,1];
c = configuration_array[:,2];
I = 10000.0; # default budget is 10K USD
bounds = [
        0  1 ; # 1 CPU 1
        0  1 ; # 2 CPU 2
        
        0  1 ; # 3 Memory 1
        0  1 ; # 4 Memory 2
        0  1 ; # 5 Memory 3

        0  1 ; # 6 Storage 1
        0  1 ; # 7 Storage 2
        0  1 ; # 8 Storage 3
        0  1 ; # 9 Storage 4

        0  1 ; # 10 Accessory 1 
        0  1 ; # 11 Accessory 2
        0  1 ; # 12 Accessory 3
        
        1  1 ; # 13 Software 1
        1  1 ; # 14 Software 2
    ]; #choose/not choose a feature

# build the problem object -
problem = build(MySimpleBinaryVariableLinearChoiceProblem, (
    
    initial = zeros(14), 
    α = α, # coefficients
    c = c, # costs
    I = I, # default budget is 10K USD
    C = C, # constraint matrix
    # 
    bounds = bounds 
));

solution = solve(problem);
solution

Dict{String, Any} with 3 entries:
  "argmax"          => [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, …
  "budget"          => 8447.0
  "objective_value" => 2.416

#### Check: Are the choice constraints enforced?
We can only select a fixed number of items from each category. Does your solution reflect this restriction?

In [21]:
# the expected constraint vector
expected_vector = [1, 1, 1, 1]

x = solution["argmax"]
result_vector = C * x
is_correct = all(result_vector .== expected_vector)
if is_correct
    println("The solution respects the choice constraints.")
else
    println("The solution violates the choice constraints.")
end

The solution respects the choice constraints.


### Task 4: How does changing the $\alpha$-vector (or the budget $I$) influence the configuration choice?

In [28]:
# when budget increases, the solution remains the same (cause the budget was not used up in the initial solution)
I = 20000.0;

problem = build(MySimpleBinaryVariableLinearChoiceProblem, (
    
    initial = zeros(14), 
    α = α, # coefficients
    c = c, # costs
    I = I, # default budget is 20K USD
    C = C, # constraint matrix
    # 
    bounds = bounds 
));

solution = solve(problem);
solution
#solution["argmax"]

Dict{String, Any} with 3 entries:
  "argmax"          => [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, …
  "budget"          => 8447.0
  "objective_value" => 2.416

In [26]:
# when there's a decrease in budget and it's now below 8447.0 (which is the original amount of money spent)
I = 8000.0;

problem = build(MySimpleBinaryVariableLinearChoiceProblem, (
    
    initial = zeros(14), 
    α = α, # coefficients
    c = c, # costs
    I = I, # default budget is 8K USD
    C = C, # constraint matrix
    # 
    bounds = bounds 
));

solution = solve(problem);
solution

Dict{String, Any} with 3 entries:
  "argmax"          => [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, …
  "budget"          => 7807.0
  "objective_value" => 2.166

In [27]:
# the consumer will choose Memory 1 instead of Memory 2 in this case, and opt out the Storage 
solution["argmax"]

14-element Vector{Float64}:
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0
 1.0

In [32]:
# preference change
# assume stronger preferences towards the more/most expensive features
configuration_array = [

    # CPU options
    0.1 2640.0    ; # 1 CPU 1
    0.9 3649.0    ; # 2 CPU 2

    # Memory options
    0.1 3840.0  ; # 3 Memory 1
    0.8 4640.0  ; # 4 Memory 2
    0.1 3600.0  ; # 5 Memory 3

    # Storage options -
    0.1 1440.0   ; # 6 Storage 1
    0.1 1840.0   ; # 7 Storage 2
    0.1 2440.0   ; # 8 Storage 3
    0.7 3640.0   ; # 9 Storage 4

    # Accessory options
    0.1 79.0    ; # 10 Accessory 1
    0.1 129.0   ; # 11 Accessory 2
    0.8 149.0   ; # 12 Accessory 3
    
    # Software options
    0.5 299.0     ; # 13 Software 1
    0.5 149.0     ; # 14 Software 2
];

α = configuration_array[:,1];
I = 10000.0;
problem = build(MySimpleBinaryVariableLinearChoiceProblem, (
    
    initial = zeros(14), 
    α = α, # coefficients
    c = c, # costs
    I = I, # default budget is 10K USD
    C = C, # constraint matrix
    # 
    bounds = bounds 
));

solution = solve(problem);
solution

# the consumer will end up spend more money on the MacBook Pro

Dict{String, Any} with 3 entries:
  "argmax"          => [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, …
  "budget"          => 8886.0
  "objective_value" => 3.5